## AIT526-S01 Programming Assignment 1 - Chatbot Eliza
---------------------------------------------------------------------------------------

Created by Dr. Liao (6/5/2020)

Professor Maryam Heidari

---------------------------------------------------------------------------------------
Group 4

Students' Full Name: Penny O'Brien, Maniphone Sourivong, Muge Yalcin

Submission Date: 6/10/2023

---------------------------------------------------------------------------------------
### Programming Assignment - Chatbot

##### Import libraries

In [1]:
import re
import random
import tkinter as tk
from tkinter import ttk
from nltk.tokenize import RegexpTokenizer

##### Create conversation starter

In [2]:
CONVERSATION_STARTER = "Hi, I'm a psychotherapist. What is your name?"
USER_NAME = ""

##### Create rules to match words with corresponding responses

In [3]:
RULES = {
    r".*name.*\b(\w+)$": {'type': 'name', 'responses': ['Hi {name} ! How can I help you today?', 'It is very nice to meet you {name}, how may I help you?']},
    r".*suicid.*|.*kill.*": {'type': 'alert', 'responses': ["If you're feeling suicidal, please seek help from a trusted adult or call a helpline."]},
    r"\byes\b|\bno\b|\bright\b|\bnope\b": {'type': 'short_ans', 'responses': ['Tell me more.', 'Can you expand on that?', 'Are you sure?', "Are you comfortable in telling me more about it?"]},
    r".*want(.*)": {'type': 'want', 'responses': ["Tell me more about what you want.", "Why do you want{}?", "What would you feel if you get{}?"]},
    r".*crave (.*)|.*craving (.*)": {'type': 'want', 'responses': ["Tell me more about your cravings.", "What specifically are you craving?", "Why are you crawing {}"]},
    r".*i am(.*)|.*i'm(.*)|.*i have been(.*)": {'type': 'am', 'responses': ["How long have you been{}?","Why do you think you are{}?", "How does being{} make you feel?", "Why are you{}?"]},
    r".*because(.*)": {'type': 'explain', 'responses': ["Really? Just because{}?"]},
    r".*dunno.*|.*idk.*|.*i don.t know.*|.*i dont know.*": {'type': 'explain', 'responses': ["Maybe you know, can you try to explain?", "Please try your best to explain?"]},
    r".*feel(.*)": {'type': 'feels', 'responses': ["What is making you feel{}?", "Why do you think you feel{}?", "Since when have you been feeling{}?"]},
    r".*think(.*)": {'type': 'feels', 'responses': ["Why do you think that {}?"]},
    r"(.*thanks.*)": {'type': 'acknowledgment', 'responses': ["No problem! Is there anything else you need?", "I'm glad to help {name}. Anything else I can help you with?", "You're welcome {name}! What else can I do for you?"]},
    r".*(sorry.*)": {'type': 'feels', 'responses': ["Don't worry about it. You don't need to apologize"]},
    r"(.*how are you.*)": {'type': 'question', 'responses': ["I'm doing fine, thank you! How about you?"]},
    r"(.*what is your name.*)": {'type': 'question', 'responses': ["My name is Eliza, pleased to meet you {name}!"]},
    r"exit|quit|bye|goodbye": {'type': 'exit', 'responses': ["Goodbye {name}! Take care!"]},
    r"hello|hi|hey": {'type': 'exit', 'responses': ["Hello {name}, how can I help you today?"]},
    r"^(?![\s\S])": {'type': 'none', 'responses': ["Are you there {name}?", "Hello?"]},
    r"(.*)": {'type': 'unknown', 'responses': ["I'm sorry, I didn't understand. Can you say that another way?"]}
}

##### Create response converters

In [4]:
CONVERTERS = {
    r'\bi\b|\bme\b': 'you',
    r"\bmy\b|\bour\b": 'your',
    r"\bam\b|\bm\b": 'are',
    r"\bmyself\b": 'yourself',
    r"\bim\b": 'you are',
  }

##### Tokenize input

In [5]:
tokenizer = RegexpTokenizer(r'\w+')

def clean_input(input_text):
    tokens = tokenizer.tokenize(input_text)
    tokens = [token.lower() for token in tokens]
    return " ".join(tokens)

def replace_words(response):
    for pattern, replacement in CONVERTERS.items():
        response = re.sub(pattern, replacement, response)
    return response

def capture_user_name(input_text):
    match = re.match(r".*name.*\b(\w+)", input_text)
    if match:
        return match.group(1)
    return ""

def generate_response(user_input):
    cleaned_input = clean_input(user_input)

    if cleaned_input == "exit" or cleaned_input == "quit":
        return "Farewell, take care!"

    global USER_NAME

    if not USER_NAME:
        USER_NAME = capture_user_name(cleaned_input)

    for pattern, rule in RULES.items():
        match = re.match(pattern, cleaned_input)
        if match:
            response_type = rule['type']
            responses = rule['responses']
            response = random.choice(responses)
            response = response.replace("{name}", USER_NAME)
            response = response.format(*match.groups())
            response = replace_words(response)

            return response

    return "I'm sorry, I didn't understand. Can you say that another way?"

##### Create GUI

In [6]:
def add_message(message, sender):
    chat_text.configure(state=tk.NORMAL)
    chat_text.insert(tk.END, message + "\n", sender)
    chat_text.configure(state=tk.DISABLED)
    chat_text.see(tk.END)

def send_message():
    user_input = input_text.get("1.0", "end-1c").strip()
    input_text.delete("1.0", tk.END)

    if user_input.lower() in ["exit", "quit"]:
        root.quit()

    response = generate_response(user_input)
    add_message("You: " + user_input, "You")
    add_message("Eliza: " + response, "Eliza")

# Create a GUI window
root = tk.Tk()
root.title("Chatbot")
root.configure(bg="#cffdbc")  # Set the background color

# Create a display area
chat_text = tk.Text(root, height=20, width=70)
chat_text.configure(state=tk.DISABLED)
chat_text.grid(row=0, column=0, padx=10, pady=10, sticky="nsew")

# Set the grid weights to allow the display area to expand as needed.
root.grid_rowconfigure(0, weight=1)
root.grid_columnconfigure(0, weight=1)

# Create a scrollbar
scrollbar = ttk.Scrollbar(root, orient=tk.VERTICAL, command=chat_text.yview)
scrollbar.grid(row=0, column=1, sticky="ns")
chat_text.configure(yscrollcommand=scrollbar.set)

# Create an input area
input_text = tk.Text(root, height=4, width=50)
input_text.grid(row=1, column=0, padx=10, pady=10, sticky="ew")

# Create a send button
send_button = ttk.Button(root, text="Send", command=send_message)
send_button.grid(row=1, column=1, padx=10, pady=10, sticky="e")

# Start the conversation
add_message("Eliza: " + CONVERSATION_STARTER, "Eliza")

# Run the GUI window
root.mainloop()

### References

Anon. n.d. Tkinter 8.5 Reference: A Gui for Python. Retrieved June 3, 2023 (https://tkdocs.com/shipman/). Real Python. 2023. “Python GUI Programming with Tkinter.” Real Python. Retrieved June 3, 2023 (https://realpython.com/python-gui-tkinter/).